# 🔐 Pipeline de Seguridad y Pruebas (Notebook)

Este notebook ejecuta **tres suites de pruebas**:
- **Unitarias** (objetivo ≥ 80% cobertura)
- **Seguridad** (ataques: SQLi, XSS, autenticación/rate limit)
- **Integración** (flujo `register → login → me → refresh`)

Además corre **Bandit (SAST)** y **pip-audit** (vulnerabilidades).

⚠️ **Colócalo en la raíz del proyecto** (donde está `requirements.txt`).


## 0) Configuración rápida
Puedes desactivar temporalmente la auditoría de dependencias si bloquea el flujo.


In [ ]:
SKIP_AUDIT = False  # cambia a True si deseas saltar pip-audit
REQ_FILE = 'requirements.txt'
print('SKIP_AUDIT =', SKIP_AUDIT)


## 1) Instalar dependencias


In [ ]:
import sys, subprocess, pathlib
req = pathlib.Path(REQ_FILE)
if not req.exists():
    raise FileNotFoundError('No se encontró requirements.txt en la raíz del proyecto.')
print('Python:', sys.executable)
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', str(req)])
print('Dependencias listas.')


## 2) SAST y Auditoría
- **Bandit**: análisis estático de seguridad.
- **pip-audit**: vulnerabilidades en dependencias.


In [ ]:
import sys, subprocess
print('>> Bandit (SAST)')
subprocess.check_call([sys.executable, '-m', 'bandit', '-q', '-r', 'app', '-x', 'tests'])
print('Bandit OK')

if SKIP_AUDIT:
    print('Saltando pip-audit por configuración (SKIP_AUDIT=True)')
else:
    print('>> pip-audit (dependencias) --strict')
    subprocess.check_call([sys.executable, '-m', 'pip_audit', '--strict'])
    print('pip-audit OK')


## 3) Pruebas **Unitarias** (objetivo cobertura ≥ 80%)
Usa la configuración de `pytest.ini` (umbral de cobertura). Aquí filtramos por archivo `test_unit_*.py`.


In [ ]:
import sys, subprocess
print('>> PyTest (unitarias)')
subprocess.check_call([sys.executable, '-m', 'pytest', '-q', 'tests/test_unit_security.py'])
print('Unitarias OK')


## 4) Pruebas de **Seguridad** (SQLi, XSS, rate limit, auth)


In [ ]:
import sys, subprocess
print('>> PyTest (seguridad)')
subprocess.check_call([sys.executable, '-m', 'pytest', '-q', 'tests/test_security_attacks.py'])
print('Seguridad OK')


## 5) Pruebas de **Integración** (flujo completo)


In [ ]:
import sys, subprocess
print('>> PyTest (integración)')
subprocess.check_call([sys.executable, '-m', 'pytest', '-q', 'tests/test_integration_flow.py', 'tests/test_auth.py'])
print('Integración OK')


## 6) Reporte de cobertura HTML
Generado por `pytest.ini` como `coverage_html/`. Puedes abrir `coverage_html/index.html`.
